# Prerequisites
본 `ipynb` 은 `Python=3.12` 에서 작성하였습니다. Package dependency 를 해결하기 위해 아래 cell 을 실행해주세요.

## Install Python packages

In [2]:
%pip -q install -U azure-identity azure-search-documents azure-ai-documentintelligence langchain langchain-community langchain-openai mcp

Note: you may need to restart the kernel to use updated packages.


## Load environment variables from a .env file
secret 노출을 피하고 notebook 들간의 일관된 환경변수를 설정하기 위해 `dotenv` 을 이용한다.

In [3]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
AZURE_AI_SEARCH_ENDPOINT = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
AZURE_AI_SEARCH_ADMIN_KEY = os.getenv("AZURE_AI_SEARCH_ADMIN_KEY")
AZURE_DOCUMENTINTELLIGENCE_ENDPOINT = os.getenv("AZURE_DOCUMENTINTELLIGENCE_ENDPOINT")
AZURE_DOCUMENTINTELLIGENCE_API_KEY = os.getenv("AZURE_DOCUMENTINTELLIGENCE_API_KEY")
ACCUWEATHER_API_KEY = os.getenv("ACCUWEATHER_API_KEY")
MCP_WEATHER_URL = os.getenv("MCP_WEATHER_URL")

# MCP
MCP 는 AI agent 의 표준 stack 에서 data layer 의 복잡성을 줄여 model context 에 접근하게 하는 표준 통신 규격이다.

## Communicating with MCP server by HTTP gateway

In [17]:
from mcp.client.sse import sse_client
from mcp.client.session import ClientSession

async with sse_client(MCP_WEATHER_URL + "/sse") as (r, w), ClientSession(r, w) as session:
    await session.initialize()
    data = await session.list_tools()
    
    for item in data:
        if item[0] == "tools":
            for tool in item[1]:
                print(f"name: {tool.name}")
                print(f"description: {tool.description}")
                print(f"inputSchema: {tool.inputSchema}\n")

name: weather-get_hourly
description: Get hourly weather forecast for the next 12 hours
inputSchema: {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city or location for which to retrieve the weather forecast.'}, 'units': {'type': 'string', 'description': 'Temperature unit system (metric for Celsius, imperial for Fahrenheit). Default is metric.', 'enum': ['metric', 'imperial']}}, 'required': ['location']}

name: weather-get_daily
description: Get daily weather forecast for up to 15 days
inputSchema: {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city or location for which to retrieve the weather forecast.'}, 'days': {'type': 'number', 'description': 'Number of days to forecast (1, 5, 10, or 15). Default is 5.', 'enum': [1, 5, 10, 15]}, 'units': {'type': 'string', 'description': 'Temperature unit system (metric for Celsius, imperial for Fahrenheit). Default is metric.', 'enum': ['metric', 'imperial']}}, 'required

In [6]:
import json

from mcp.client.sse import sse_client
from mcp.client.session import ClientSession
from openai import AzureOpenAI

client = AzureOpenAI(
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
)

async with sse_client(MCP_WEATHER_URL + "/sse") as (r, w), ClientSession(r, w) as session:
    await session.initialize()
    tools = await session.list_tools()
    
    tools = [{
        "type": "function",
        "function": {
            "name": tool.name,
            "description": tool.description,
            "parameters": tool.inputSchema,
        }
    } for tool in tools.tools]
    while True:
        city = input("도시 이름을 입력하세요: ")
        if not city or city.lower() == "exit":
            break
    
        messages = [{"role": "user", "content": f"{city}의 현재 날씨를 알려줘."}]
        r = client.chat.completions.create(
            model="gpt-5", messages=messages, tools=tools,
        )
        
        msg = r.choices[0].message
        messages.append(msg)

        # Handle function calls
        if msg.tool_calls:
            for tool_call in msg.tool_calls:
                    print("=== 🤖 Tool Calling Message ===")
                    print(f"Tool Name: {tool_call.function.name}")
                    print(f"Tool Arguments: {tool_call.function.arguments}")

                    function_args = json.loads(tool_call.function.arguments)
                    result = await session.call_tool(tool_call.function.name, function_args)
                    messages.append({
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": tool_call.function.name,
                        "content": result.content,
                    })
        else:
            print("No tool calls were made by the model")

        # Get the final response from the model
        r = client.chat.completions.create(
            model="gpt-5", messages=messages, tools=tools,
        )

        print("=== 🤖 Response Message ===")
        print(r.choices[0].message.content)
        print("\n=== 🤖 Response Usage ===")
        print(r.usage.model_dump_json(indent=2))

=== 🤖 Tool Calling Message ===
Tool Name: weather-get_hourly
Tool Arguments: {"location":"Paris, France","units":"metric"}
=== 🤖 Response Message ===
파리 현지시각 23:00 기준 현재 기온은 약 13.4°C이며, 대체로 흐립니다.

=== Response Usage ===
{
  "completion_tokens": 679,
  "prompt_tokens": 618,
  "total_tokens": 1297,
  "completion_tokens_details": {
    "accepted_prediction_tokens": 0,
    "audio_tokens": 0,
    "reasoning_tokens": 640,
    "rejected_prediction_tokens": 0
  },
  "prompt_tokens_details": {
    "audio_tokens": 0,
    "cached_tokens": 0
  }
}
